# Extracting Data from Marktstammdatenregister for further examination 
renewable generation facilities (Wind, solar) located in 50Hertz-grid that received funds from renewbale support regime (Erneuerbare Energien Gesetz EEG)

In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## total capacity wind and solar 50 Hz aggregates per Bundesland

In [66]:
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\Kraftwerksliste_CSV_utf.csv"
plants = pd.read_csv(data, sep=";", encoding="utf8")
wind_solar_50_Hz = plants[(
                (plants["Bundesland der Einheit"]=="Hamburg") |
                (plants["Bundesland der Einheit"]=="Mecklenburg-Vorpommern") |
                (plants["Bundesland der Einheit"]=="Brandenburg") |
                (plants["Bundesland der Einheit"]=="Berlin") |
                (plants["Bundesland der Einheit"]=="Sachsen-Anhalt") |
                (plants["Bundesland der Einheit"]=="Thueringen") |
                (plants["Bundesland der Einheit"]=="Sachsen")
                )
                &
                (
                (plants["Energieträger"] == "Solare Strahlungsenergie") |
                (plants["Energieträger"] == "Windenergie (Onshore-Anlage)") |
                (plants["Energieträger"] == "Windenergie (Offshore-Anlage)")
                )]

## EEG-funded facilities in 50 HZ grid 2021

In [ ]:
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\50Hertz Transmission GmbH EEG-Zahlungen Bewegungsdaten 2021.csv"
EEG_2021_50_hz = pd.read_csv(data, sep=";")

In [ ]:
EEG_2021_50_hz.info()

In [ ]:
EEG_2021_50_hz.EEG_Mastr_Nr.nunique()

not all facilities listed have EEG_Mastr_Nr

In [ ]:
EEG_2021_50_hz[EEG_2021_50_hz.EEG_Mastr_Nr.isna()]

actual generation and installed generation capacity for total 50 HZ grid per hour in 2020/21

In [ ]:
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\Realisierte_Erzeugung_50_Hertz_2020_2021_Hour.csv"
generation_50_hz = pd.read_csv(data, sep=";")
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\Installierte_Erzeugungsleistung_50_Hertz_2020_2021_Hour.csv"
capacity_50_hz = pd.read_csv(data, sep=";")

spotmarket_prices electricity EEX, very low and even negative prices indicate abundance of renewable electricity generation

In [ ]:
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\new_csv\spot_market_2021_onwards.csv"
spot_market = pd.read_csv(data)

PV in 50 HZ zone

In [ ]:
def mastr_import_fac(n):
    data = rf"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\MaStR\EinheitenSolar_{n}.xml"
    solar_fac = pd.read_xml(data, encoding="utf-16")
    solar_fac_50_hz = solar_fac[solar_fac.EegMaStRNummer.isin(EEG_2021_50_hz.EEG_Mastr_Nr)]
    return solar_fac_50_hz

solar_fac_50hz = []
for i in range(1,30):
    solar_fac_50hz.append(mastr_import_fac(i))
pv_fac_50_hz = pd.concat(solar_fac_50hz)

In [ ]:
pv_fac_50_hz.info()

In [ ]:
def mastr_import(n):
    data = rf"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\MaStR\AnlagenEegSolar_{n}.xml"
    solar = pd.read_xml(data, encoding="utf-16")
    solar_50_hz = solar[(solar.EegMaStRNummer.isin(EEG_2021_50_hz.EEG_Mastr_Nr)) | (solar.AnlagenschluesselEeg.isin(EEG_2021_50_hz["EEG_Anlagenschlüssel"]))]
    return solar_50_hz

solar_50hz = []
for i in range(1,30):
    solar_50hz.append(mastr_import(i))
pv_50_hz = pd.concat(solar_50hz)

In [ ]:
pv_50_hz.info()

In [ ]:
pv = pv_50_hz.copy()

In [ ]:
pv_50_hz.EegMaStRNummer.nunique()

In [ ]:
## filtering raw data by "Anlagenschlüssel"
def mastr_import(n):
    data = rf"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\MaStR\AnlagenEegSolar_{n}.xml"
    solar = pd.read_xml(data, encoding="utf-16")
    solar_50_hz = solar[solar.AnlagenschluesselEeg.isin(EEG_2021_50_hz["EEG_Anlagenschlüssel"])]
    return solar_50_hz

solar_50hz = []
for i in range(1,30):
    solar_50hz.append(mastr_import(i))
anlagenschluessel_pv_50_hz = pd.concat(solar_50hz)

In [ ]:
anlagenschluessel_pv_50_hz.info()

In [ ]:
pv_50_hz.AnlagenschluesselEeg.nunique()

In [ ]:
pv_50_hz.EegMaStRNummer.nunique()

In [ ]:
pv_50_hz.to_csv("solar_50_hz.csv", index=False)

In [ ]:
pv_50_hz.AnlagenschluesselEeg.nunique()

In [ ]:
anlagenschluessel_pv_50_hz.EegMaStRNummer.nunique()

In [ ]:
anlagenschluessel_pv_50_hz.AnlagenschluesselEeg.nunique()

In [ ]:
pv_50_hz[(pv_50_hz.AnlagenschluesselEeg.isna()==False) & (pv_50_hz.)]

In [ ]:
#testing pv_fac with mastr-numbers from pv-50-hz instead of 50-hz-eeg 
def mastr_import_fac(n):
    data = rf"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\MaStR\EinheitenSolar_{n}.xml"
    solar_fac = pd.read_xml(data, encoding="utf-16")
    solar_fac_50_hz = solar_fac[solar_fac.EegMaStRNummer.isin(pv_50_hz.EegMaStRNummer)]
    return solar_fac_50_hz

solar_fac_50hz = []
for i in range(1,30):
    solar_fac_50hz.append(mastr_import_fac(i))
fac_pv_50_hz = pd.concat(solar_fac_50hz)

In [ ]:
fac_pv_50_hz.info()

In [ ]:
fac_pv_50_hz[fac_pv_50_hz.EinheitMastrNummer.isin(pv_fac_50_hz.EinheitMastrNummer)==False]

## short glimpse on those rows show, that they are (at least partially) located in eastern germany, so the DF extracted based on pv_50_hz instead of EEG_50_hz is kept and extracted

In [ ]:
fac_pv_50_hz.to_csv("solar_50hz_facilities.csv", index=False)

biomass in 50 HZ grid

In [ ]:
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\MaStR\EinheitenBiomasse.xml"
biomass_fac = pd.read_xml(data, encoding="utf-16")
biomass_fac_50_Hz = biomass_fac[biomass_fac.EegMaStRNummer.isin(EEG_2021_50_hz.EEG_Mastr_Nr)]

In [ ]:
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\MaStR\AnlagenEegBiomasse.xml"
biomass = pd.read_xml(data, encoding="utf-16")
biomass_50_hz = biomass[biomass.EegMaStRNummer.isin(EEG_2021_50_hz.EEG_Mastr_Nr)]

wind in 50 Hz grid

In [ ]:
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\MaStR\EinheitenWind.xml"
wind_fac = pd.read_xml(data, encoding="utf-16")
wind_fac_50_hz = wind_fac[wind_fac.EegMaStRNummer.isin(EEG_2021_50_hz.EEG_Mastr_Nr)]

In [ ]:
wind_fac_50_hz.info()

In [ ]:
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\MaStR\AnlagenEegWind.xml"
wind = pd.read_xml(data, encoding="utf-16")
wind_50_hz = wind[wind.EegMaStRNummer.isin(EEG_2021_50_hz.EEG_Mastr_Nr)]

In [ ]:
wind_50_hz.info()

In [ ]:
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\MaStR\AnlagenEegWind.xml"
wind = pd.read_xml(data, encoding="utf-16")
anlagenschlüssel_wind_50_hz = wind[wind.AnlagenschluesselEeg.isin(EEG_2021_50_hz["EEG_Anlagenschlüssel"])]

In [ ]:
anlagenschlüssel_wind_50_hz.info()

In [ ]:
## wind combined
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\MaStR\AnlagenEegWind.xml"
wind = pd.read_xml(data, encoding="utf-16")
combined_wind_50_hz = wind[(wind.AnlagenschluesselEeg.isin(EEG_2021_50_hz["EEG_Anlagenschlüssel"])) | (wind.EegMaStRNummer.isin(EEG_2021_50_hz.EEG_Mastr_Nr))]



In [ ]:
combined_wind_50_hz.info()

In [ ]:
combined_wind_50_hz.to_csv("wind_50_hz.csv", index=False)

In [ ]:
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\MaStR\EinheitenWind.xml"
fac_wind = pd.read_xml(data, encoding="utf-16")
fac_wind_50_hz = fac_wind[fac_wind.EegMaStRNummer.isin(combined_wind_50_hz.EegMaStRNummer)] 

In [ ]:
fac_wind_50_hz.info()

In [ ]:
fac_wind_50_hz.to_csv("wind_50_hz_facilities.csv", index=False)

many qualitative columns are encoded in numeric values, importing the "catalogue_values"

In [ ]:
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\MaStR\Katalogwerte.xml"
catalogue_values = pd.read_xml(data, encoding="utf-16")